# Model Selection

In this notebook we will test different models to find the one that gives us the best results.

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def f1_scores(Z, y_test):
    averages = ['macro', 'micro', 'weighted']
    for avg in averages:
        score = f1_score(Z, y_test, average=avg)
        print("f1 score ({}): {}".format(avg, score))

def test_model(X, y, model_name, model):
    print("MODEL: {}".format(model_name))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model.fit(X_train, y_train)
    Z = model.predict(X_test)
    f1_scores(Z, y_test)
    
def testModelKFold(X, y, model_name, model, k):
    print("CROSS VALIDATION FOR: {}".format(model_name))
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(X, y)
    f1_scores = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        Z = model.predict(X_test)
        f1_scores.append(f1_score(Z, y_test))
    print("f1 scores: {}".format(f1_scores))
    print("MEAN: {}".format(np.mean(f1_scores)))

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

models = {
    "Logistic Regression": LogisticRegression(),
    "SVC": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors = 3),
    "GaussianNB": GaussianNB(),
    "Perceptron": Perceptron(),
    "SGDClassifier": SGDClassifier(),
    "Decision Tree": DecisionTreeClassifier()   
}

dataset_path = "data-all.csv"
data = pd.read_csv(dataset_path, sep=";")
X = data.drop("diagnosis", axis=1).values
y = data["diagnosis"]

for model in models:
    test_model(X, y, model, models[model])
    testModelKFold(X, y, model, models[model], k=5)
    print('')

MODEL: Logistic Regression
f1 score (macro): 0.8148019900852963
f1 score (micro): 0.8531889290012034
f1 score (weighted): 0.8554887632270022
CROSS VALIDATION FOR: Logistic Regression
f1 scores: [0.8992805755395683, 0.8727755644090306, 0.8852734922861151, 0.9055429864253393, 0.8699576868829337]
MEAN: 0.8865660611085975

MODEL: SVC
f1 score (macro): 0.8371379904226619
f1 score (micro): 0.8752507019655034
f1 score (weighted): 0.8820452566767064
CROSS VALIDATION FOR: SVC
f1 scores: [0.9262013729977117, 0.8781127129750982, 0.8931193908077236, 0.8995002776235425, 0.8878706199460916]
MEAN: 0.8969608748700335

MODEL: Random Forest
f1 score (macro): 0.838970112844251
f1 score (micro): 0.8712394705174489
f1 score (weighted): 0.8747382091182168
CROSS VALIDATION FOR: Random Forest
f1 scores: [0.906074074074074, 0.8914410719168717, 0.8905150576977202, 0.8728686808256058, 0.873247335950645]
MEAN: 0.8868292440929834

MODEL: K-Nearest Neighbors
f1 score (macro): 0.7955462302245527
f1 score (micro): 0.

We can check the confusion matrix for the Random Forest model.

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model = RandomForestClassifier(n_estimators=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
Z = model.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, Z))

print("\nClassfication Report")
print(classification_report(y_test, Z))

Confusion Matrix
[[ 547  221]
 [ 109 1616]]

Classfication Report
             precision    recall  f1-score   support

        0.0       0.83      0.71      0.77       768
        1.0       0.88      0.94      0.91      1725

avg / total       0.87      0.87      0.86      2493



## Save model

In [10]:
import dill as pickle

def dump_model(model, path):
    with open(path, 'wb') as file:
        pickle.dump(model, file)
        
# Define path to save the model file
filename = "model_v2.pk"
dir_path = "./"

# Dump model to file
dump_model(model, dir_path + filename)